## Load library modules

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

default_dir = os.path.dirname(os.path.dirname(os.getcwd()))
os.chdir(default_dir)

# (Mac) Uncomment below two lines if there's an error with PyJulia setup (initial execution may take some time)
# from julia.api import Julia
# jl = Julia(compiled_modules=False)

from modWorm import sys_paths
from modWorm import network_params as n_params
from modWorm import network_dynamics as n_dyn
from modWorm import network_interactions as n_inter
from modWorm import network_simulations as n_sim

from modWorm import muscle_body_params as mb_params
from modWorm import muscle_dynamics as m_dyn
from modWorm import body_dynamics as b_dyn
from modWorm import body_simulations as b_sim

from modWorm import proprioception_simulation as p_sim

from modWorm import utils
from modWorm import animation

### Load experimental connectome data
- Varshney, Lav R., et al. "Structural properties of the Caenorhabditis elegans neuronal network." PLoS computational biology 7.2 (2011): e1001066.
- Haspel, Gal, and Michael J. O'Donovan. "A perimotor framework reveals functional segmentation in the motoneuronal network controlling locomotion in Caenorhabditis elegans.
- Download link: https://www.wormatlas.org/images/NeuronConnect.xls
- Install openpyxl via "conda install openpyxl" or "pip install openpyxl" if prompted with dependency error

### Load neurons to muscles mapping data
- WormAtlas, Altun, Z.F., Herndon, L.A., Wolkow, C.A., Crocker, C., Lints, R. and Hall, D.H. (ed.s) 2002-2024.
- Download link: https://www.wormatlas.org/images/NeuronFixedPoints.xls
- If the file is in old .xls format, make sure to save as .xlsx format

In [ ]:
# Construct gap, synaptic connectomes and muscle map from downloaded files
conn_gap, conn_syn = utils.construct_connectome_Varshney(filepath)
muscle_map = utils.construct_muscle_map_Hall(filepath)

## Define Nervous System

In [ ]:
from modWorm import predefined_classes_nv, predefined_classes_mb

celegans_nv = predefined_classes_nv.CelegansWorm_NervousSystem_PPC_Julia(conn_gap, conn_syn)
celegans_mb = predefined_classes_mb.CelegansWorm_MuscleBody_PPC_Julia(muscle_map)

## Define Stimulation

In [ ]:
# We use the pulse of neural stimuli emulating the gentle touch to C.elegans posterior body region
# input_mat has dim = (1400, 279) (14 seconds)

gentle_posterior_stim = np.load('modWorm/presets_input/input_mat_gentle_post_touch.npy')

In [ ]:
# Pre-defined stimuli array inject decaying stimuli into PLM neurons

plt.plot(np.arange(0, 14, 0.01)[:400], gentle_posterior_stim[:400, 276])
plt.xlabel('Seconds')
plt.ylabel('pA')

## Simulate Nervous System

In [ ]:
# Use run_network() function from "proprioception_simulation" module NOT "network_simulation" module

solution_dict_fwd = p_sim.run_network_julia(celegans_nv, celegans_mb, gentle_posterior_stim)

In [ ]:
solution_dict_fwd.keys()

## Analyze results

In [ ]:
# Plot the body trajectory

plt.plot(solution_dict_fwd['x_solution'][:, 0], solution_dict_fwd['y_solution'][:, 0])
plt.xlim(-75, 75)
plt.ylim(-75, 75)

## Animate body simulation

In [ ]:
animation.animate_body(x = solution_dict_fwd['x_solution'], y = solution_dict_fwd['y_solution'], filename = 'fwd_locomotion_ppc',
                       xmin = -70, xmax = 20, ymin = -20, ymax = 70,
                       figsize_x = 10, figsize_y = 10, 
                       background_img_path = False, animation_config = mb_params.CE_animation)